In [2]:
import os
import pandas as pd
from tqdm import tqdm
from tqdm.notebook import tqdm
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI




# Model setup ..
def gen_content_with_openai(prompt):
    model_name = "gpt-4-1106-preview"
    temperature = 0.2
    model = OpenAI(model_name=model_name, temperature=temperature)
    output = model(prompt)
    return output

# loading questions ..
df = pd.read_csv('files/test.csv')
df = df[['question', 'context', 'answers', 'correct_answer_id']]
paragraphs = df['context'].unique()
len(paragraphs)

##  Prompt for generating questions ..
def generate_prompt(text, bloom_category):
    prompt = f"Given the following paragraph: '{text}', generate some questions that best fits the Bloom's Taxonomy category '{bloom_category}'.\n\n\
    Sample Examples and Guidance for '{bloom_category}' category:\n\
    - Remembering: Questions in this category involve recalling factual information without necessarily understanding its significance. Example: 'What is the capital city of France?'\n\
    - Understanding: Questions in this category require comprehending or grasping the meaning of information. Example: 'Explain the concept of photosynthesis in plants.'\n\
    - Applying: Questions in this category involve using acquired knowledge in new situations or applying learned concepts in different contexts. Example: 'If a car travels at a speed of 60 km/h for 2 hours, how far does it travel?'\n\
    - Analyzing: Questions in this category involve breaking down information into parts and examining the relationships between them. Example: 'Compare and contrast the features of a laptop and a tablet.'\n\
    - Evaluating: Questions in this category involve making judgments or assessments based on criteria and standards. Example: 'Which of the following solutions is the most effective for reducing traffic congestion in cities?'\n\
    - Creating: Questions in this category involve generating new ideas, solutions, or products. Example: 'Design a new product that solves a common problem in daily life.'"
    return prompt


##  Generating answers from gemini - 
df = pd.read_csv('question.csv')
df

,Bloom's Taxonomy Category,Question,gemini_answers,paragraphs
0,Remembering,Where did the narrator set their car keys down...,NaN,The biggest thorn in the paw of my grand plan ...
1,Remembering,What was the reason Cathy gave for not attendi...,NaN,The biggest thorn in the paw of my grand plan ...
2,Understanding,Why does the narrator think their mom is going...,NaN,The biggest thorn in the paw of my grand plan ...
3,Understanding,"What does the phrase ""a bit batty"" suggest abo...",NaN,The biggest thorn in the paw of my grand plan ...
4,Application,How might the narrator's situation (mom leavin...,NaN,The biggest thorn in the paw of my grand plan ...
...,...,...,...,...
67,Analysis,How does the detail about the paramedic offeri...,NaN,"It was cold, it was dark and it was raining th..."
68,Evaluation,Do you think the paramedic's approach to the s...,NaN,"It was cold, it was dark and it was raining th..."
69,Evaluation,How does the passage portray the challenges of...,NaN,"It was cold, it was dark and it was raining th..."
70,Creative,Write a diary entry from the paramedic's persp...,NaN,"It was cold, it was dark and it was raining th..."


In [3]:
"""def generate_question_dataframe(paragraphs, filename='questions.csv'):
    bloom_cats = ['Remembering', 'Understanding', 'Analyzing', 'Application', 'Evaluation', 'Higher Order thinking Skills']

    data = []

    for item in tqdm(paragraphs, desc='Paragraphs'):
        for bloom_cat in tqdm(bloom_cats, desc='Bloom Category', leave=False):
            questions = generate_questions(item, bloom_cat)
            data.append({'questions':questions, 'bloom_cat':bloom_cat, 'paragraph':item})
    return data

# Example usage:
data = generate_question_dataframe(paragraphs[0:5])
data"""

"def generate_question_dataframe(paragraphs, filename='questions.csv'):\n    bloom_cats = ['Remembering', 'Understanding', 'Analyzing', 'Application', 'Evaluation', 'Higher Order thinking Skills']\n\n    data = []\n\n    for item in tqdm(paragraphs, desc='Paragraphs'):\n        for bloom_cat in tqdm(bloom_cats, desc='Bloom Category', leave=False):\n            questions = generate_questions(item, bloom_cat)\n            data.append({'questions':questions, 'bloom_cat':bloom_cat, 'paragraph':item})\n    return data\n\n# Example usage:\ndata = generate_question_dataframe(paragraphs[0:5])\ndata"

In [5]:
# Gemini ..
import warnings, urllib3
warnings.filterwarnings('ignore')
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)


def generate_prompt(question, paragraph):
    prompt = f"Hello! I have a reading comprehension task where I need to generate answers to questions based on given paragraphs. Could you please provide me with an answer to the following question?\n\nQuestion: {question}\n\nParagraph: {paragraph}\n\nThank you!"
    return prompt

# Model setup ..
def gen_content_with_openai(prompt):
    model_name = "gpt-4-1106-preview"
    temperature = 0.2
    model = OpenAI(model_name=model_name, temperature=temperature)
    output = model(prompt)
    return output


##  Iterating thru the questions and answering ..
answers = []

count = 1
for item in df.iterrows():
    question = item[1]['Question']
    paragraph = item[1]['paragraphs']
    prompt = generate_prompt(question, paragraph)
    out = gen_content_with_openai(prompt)
    answers.append(out)
    print(count)
    print('Question : ', question)
    print('Answer : ', out)
    print('-------')
    count = count = count+1
    if count==73:
        break

1
Question :  Where did the narrator set their car keys down upon entering the house?
Answer :  The narrator set their car keys down on the kitchen table upon entering the house.
-------
2
Question :  What was the reason Cathy gave for not attending the narrator's father's funeral?
Answer :  Cathy, the narrator's aunt, did not attend the narrator's father's funeral because she said she wouldn't be able to drive herself, and nobody could pick her up.
-------
3
Question :  Why does the narrator think their mom is going to visit their Aunt Cathy?
Answer :  The narrator thinks their mom is going to visit their Aunt Cathy because Aunt Cathy is not doing well following her divorce and the death of the narrator's father. The mom wants to spend a few days with Aunt Cathy to help her feel better during this difficult time.
-------
4
Question :  What does the phrase "a bit batty" suggest about the narrator's feelings towards their Aunt Cathy?
Answer :  The phrase "a bit batty" suggests that the 

In [8]:
df = pd.read_csv('questions2.csv')
df['ChatGPT'] = answers

In [9]:
df

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,Bloom's Taxonomy Category,Question,gemini_answers,paragraphs,Zephyr_answers,llama2,ChatGPT
0,0,0,0,Remembering,Where did the narrator set their car keys down...,Kitchen table,The biggest thorn in the paw of my grand plan ...,The narrator set their car keys down on the ki...,"Of course! Based on the given paragraph, the a...",The narrator set their car keys down on the ki...
1,1,1,1,Remembering,What was the reason Cathy gave for not attendi...,Cathy gave the reason that she wouldn't be abl...,The biggest thorn in the paw of my grand plan ...,The reason Cathy gave for not attending the na...,"Of course! Based on the given paragraph, the r...","Cathy, the narrator's aunt, did not attend the..."
2,2,2,2,Understanding,Why does the narrator think their mom is going...,The narrator thinks their mom is going to visi...,The biggest thorn in the paw of my grand plan ...,The narrator thinks their mom is going to visi...,"Of course! Based on the given paragraphs, the ...",The narrator thinks their mom is going to visi...
3,3,3,3,Understanding,"What does the phrase ""a bit batty"" suggest abo...","The phrase ""a bit batty"" suggests that the nar...",The biggest thorn in the paw of my grand plan ...,"The phrase ""a bit batty"" suggests that the nar...","Sure, I'd be happy to help! Based on the parag...","The phrase ""a bit batty"" suggests that the nar..."
4,4,4,4,Application,How might the narrator's situation (mom leavin...,The narrator's situation of their mom leaving ...,The biggest thorn in the paw of my grand plan ...,The narrator's situation (mom leaving and free...,"Based on the given paragraphs, the narrator's ...","The narrator's situation, with their mom leavi..."
...,...,...,...,...,...,...,...,...,...,...
67,67,67,67,Analysis,How does the detail about the paramedic offeri...,The detail about the paramedic offering to tak...,"It was cold, it was dark and it was raining th...",The detail about the paramedic offering to tak...,The detail about the paramedic offering to tak...,The detail about the paramedic offering to tak...
68,68,68,68,Evaluation,Do you think the paramedic's approach to the s...,"Yes, the paramedic's approach to the situation...","It was cold, it was dark and it was raining th...","Based on the given paragraphs, it is clear tha...","Of course, I'd be happy to help you with your ...","Based on the provided paragraph, it appears th..."
69,69,69,69,Evaluation,How does the passage portray the challenges of...,The passage portrays the challenges of dealing...,"It was cold, it was dark and it was raining th...",The passage portrays the challenges of dealing...,Of course! Here are some ways the passage port...,The passage portrays the challenges of dealing...
70,70,70,70,Creative,Write a diary entry from the paramedic's persp...,**Paramedic's Diary Entry**\n\n**Date:** [Date...,"It was cold, it was dark and it was raining th...","Dear Diary,\n\nTonight's call was unlike any o...",Of course! Here is the diary entry from the pa...,"Dear Diary,\n\nTonight's shift has left me wit..."


In [10]:
df.to_csv("final.csv")

In [7]:
import pandas as pd

df = pd.read_csv("valid_final.csv")
df = df[df["validity"]=="Invalid"]
df = df[df["Bloom's Taxonomy Category"]=="Knowledge (Remembering)"]
df

,Bloom's Taxonomy Category,Question,validity,Extarctive Answer,Extraction score,gemini_pred for valid/invalid,gemini_answers,Human Score for correct ans per bloom cat,paragraphs,Unnamed: 9,Unnamed: 10,Unnamed: 11
12,Knowledge (Remembering),Where did the narrator's mom go for her vacation?,Invalid,Aunt Cathy,0.011716,Delaware,NaN,NaN,The biggest thorn in the paw of my grand plan ...,NaN,NaN,NaN
30,Knowledge (Remembering),What time did the accident occur?,Invalid,Two a.m.,0.028338,The paragraph does not contain the answer to t...,NaN,NaN,Two a.m. and we are standing on the side of th...,NaN,NaN,NaN
52,Knowledge (Remembering),What time did the young man start feeling his ...,Invalid,late at night,0.455681,"Yes, the young man started feeling his breathi...",NaN,NaN,The staffing of ambulances at the moment is .....,NaN,NaN,NaN
74,Knowledge (Remembering),What type of rash did the baby with possible m...,Invalid,right,0.266350,"Yes, the baby had a rash on the back of the knee.",NaN,NaN,I think I'm going to be leaving my job soon.\n...,NaN,NaN,NaN
96,Knowledge (Remembering),What type of weather was it during the incident?,Invalid,crappy,0.465675,"Yes, the weather was cold, dark, and raining.",NaN,NaN,"It was cold, it was dark and it was raining th...",NaN,NaN,NaN
